In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

#Load the model
#Just change the path accordingly, im using drive lol
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/My Drive/GenshinModel_save/')

#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('/content/drive/My Drive/GenshinModel_save/')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def test_review(review_text, model, tokenizer, max_length):
   #tokenize and preprocess the review
    encoded_review = tokenizer.encode_plus(
        review_text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    #extract token IDs and attention mask
    input_ids = encoded_review['input_ids']
    attention_mask = encoded_review['attention_mask']

    #ensure the tensors are on the appropriate device
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    #make predictions using the model
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    #get the predicted sentiment (argmax of the logits)
    predicted_sentiment = torch.argmax(logits, dim=1).item()

    #map predicted label to sentiment category (0: negative, 1: neutral, 2: positive)
    sentiment_mapping = {0: "NEGATIVE", 1: "NEUTRAL", 2: "POSITIVE"}
    predicted_label= sentiment_mapping[predicted_sentiment]

    return predicted_sentiment, predicted_label

#Ask the user for input
review_text = input("Enter your review text: ")

#Example usage
predicted_sentiment, predicted_label = test_review(review_text, model, tokenizer, max_length=64)
print("Predicted Sentiment:", predicted_sentiment)
print("Predicted Label:", predicted_label)

Enter your review text: i hate it but i also love it at the same time. You know what? I think i love it
Predicted Sentiment: 2
Predicted Label: POSITIVE
